## Hybrid turning approach

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
from gymnasium.utils.env_checker import check_env
from IPython.display import Video
from enum import Enum, auto

from flygym.mujoco import Parameters
from flygym.mujoco.arena import FlatTerrain
from flygym.mujoco.examples.obstacle_arena import ObstacleOdorArena
from flygym.mujoco.examples.turning_controller import HybridTurningNMF

## Setting up the Finite Sate Machine


In [17]:
# class movingFSM:
#     def __init__(self):
#         # Start in the 'go straight' state
#         self.state = 'go straight'
#         self.turn_threshold = 0
#         # Setting up the methods as dictionary entries simulating a switch
#         self.states = {
#             'go straight': self.go_straight,
#             'turn right': self.turn_right,
#             'turn left': self.turn_left,
#             'reach and turn': self.reach_and_turn
#         }
        
#     def go_straight(self, right_sense, left_sense):
#         if right_sense > self.turn_threshold:
#             self.state = 'turn left'
#         elif left_sense > self.turn_threshold:
#             self.state = 'turn right'
#         # Continue going straight if no matching sensation
#         else:
#             turning_bias = 0
#             self.state = 'go straight'
    
#     def turn_right(self, right_sense, left_sense):
#         if right_sense > self.turn_threshold:
#             self.state = 'reach and turn'
#         elif left_sense > self.turn_threshold:
#             self.state = 'turn right'
#         # Continue going straight if no matching sensation
#         else:
#             turning_bias = 0
#             self.state = 'go straight'

In [2]:
# decision_interval = 0.05
# run_time = 5
# num_decision_steps = int(run_time / decision_interval)
# physics_steps_per_decision_step = int(decision_interval / sim_params.timestep)

# obs_hist = []
# odor_history = []
# obs, _ = sim.reset(seed)
# for i in trange(num_decision_steps):
#     I_reshaped = obs["odor_intensity"].reshape((odor_dimensions, 2, 2))
#     I = np.average(I_reshaped, axis=1, weights=[120, 1200])

#     # Calculate the left-right asymmetry in the odor intensities
#     I_l, I_r = I[:, 0], I[:, 1]
#     denom = (I_l + I_r) / 2
#     denom[denom == 0] = 1  # Avoid division by zero
#     delta_I = (I_l - I_r) / denom

#     # Calculate the weighted sum of the asymmetries for each odor
#     s = np.dot(gains, delta_I)

#     # Calculate the turning bias
#     b = np.tanh(s**2)

#     control_signal = np.ones((2,))
#     side_to_modulate = int(s > 0)
#     modulation_amount = b * 0.8
#     control_signal[side_to_modulate] -= modulation_amount

#     for j in range(physics_steps_per_decision_step):
#         obs, _, _, _, _ = sim.step(control_signal)
#         rendered_img = sim.render()
#         if rendered_img is not None:
#             # record odor intensity too for video
#             odor_history.append(obs["odor_intensity"])
#         obs_hist.append(obs)


## Setting up the simulation

In [2]:
import flygym.mujoco
from tqdm import trange

# We start by creating a simple arena
flat_terrain_arena = FlatTerrain()

# Then, we add visual and olfactory features on top of it
arena = ObstacleOdorArena(
    terrain=flat_terrain_arena,
    obstacle_positions=np.array([(7.5, 0), (12.5, 5), (17.5, -5)]),
    marker_size=0.5,
    obstacle_colors=[(0.14, 0.14, 0.2, 1), (0.2, 0.8, 0.2, 1), (0.2, 0.2, 0.8, 1)],
    user_camera_settings=((13, -18, 9), (np.deg2rad(65), 0, 0), 45),
)

contact_sensor_placements = [
    f"{leg}{segment}"
    for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]

run_time = 1
sim_params = flygym.mujoco.Parameters(
    timestep=1e-4, 
    render_mode="saved", 
    render_playspeed=0.1, 
    draw_contacts=False,
    render_camera="user_cam"
)

nmf = HybridTurningNMF(
    sim_params=sim_params,
    init_pose="stretch",
    spawn_pos=(13, -5, 0.2),
    spawn_orientation=(0, 0, np.pi / 2 + np.deg2rad(70)),
    contact_sensor_placements=contact_sensor_placements,
    arena=arena
)

In [3]:
# #contact_forces in x,y,z direction of all the segments(legs) in contact sensory placements (nmf.contact_sensor_placements)

# #This gives back the contact forces of the five segments of the left and right leg 
# #without the segment touching the ground

# sensory_feedback_left = np.array([obs["contact_forces"][:5, :] for obs in obs_hist])
# sensory_feedback_right = np.array([obs["contact_forces"][18:23, :] for obs in obs_hist])


# #summed up contact forces of five of the six segments of the legs (left and right)
# #axis = 1 to sum up each row of the contact forces of the five segments of the legs
# #axis = 0 would sum up each column of the contact forces of the five segments of the legs

# sensory_feedback_sum_left = sensory_feedback_left.sum(axis=1)
# sensory_feedback_sum_right = sensory_feedback_right.sum(axis=1)

# sensory_feedback = np.array([obs["contact_forces"] for obs in obs_hist])

## Running simulation

In [4]:
# obs, info = nmf.reset()
# for i in trange(int(run_time / nmf.sim_params.timestep)):
#     curr_time = i * nmf.sim_params.timestep
#     if curr_time < 1:
#         action = np.array([1.2, 0.2])
#     else:
#         action = np.array([0.2, 1.2])

#     obs, reward, terminated, truncated, info = nmf.step(action)
#     nmf.render()

# nmf.save_video("./outputs/pillars.mp4")
# Video("./outputs/pillars.mp4")

## Reaching and turning


In [ ]:
def reach_and_turn():
    

## Running simulation with turning


In [5]:
from enum import Enum, auto

# random state seed for reproducibility
seed = 1

class State(Enum):
    GO_STRAIGHT = 1
    TURN_LEFT = 2
    TURN_RIGHT = 3
    REVERSE_LEFT = 4
    REVERSE_RIGHT = 5
    REACH_AND_TURN = 6

# Example of setting a current state
current_state = State.GO_STRAIGHT

decision_interval = 0.2
run_time = 2
num_decision_steps = int(run_time / decision_interval)
physics_steps_per_decision_step = int(decision_interval / sim_params.timestep)

low_force_thresh = 2
high_force_thresh = 5
enforce_time = 0
delay = 0.2

obs_hist = []
odor_history = []
obs, _ = nmf.reset(seed)
bias = np.array([0,0])
for i in trange(int(run_time / nmf.sim_params.timestep)):
    curr_time = i * nmf.sim_params.timestep
    left_sense = np.array(obs["contact_forces"][:5, 0:2])
    right_sense = np.array(obs["contact_forces"][18:23, 0:2])
    left_sense_sum = left_sense.sum()
    right_sense_sum = right_sense.sum()
    orig =False

    if state 

    if right_sense_sum > low_force_thresh:
        if current_state == State.GO_STRAIGHT:
            if right_sense_sum > high_force_thresh:
                current_state = State.REVERSE_LEFT
                # reverse while turning left function
            else:
                current_state = State.TURN_LEFT
                bias = np.array([-1, 0])
        elif current_state == State.TURN_LEFT:
            current_state = State.REVERSE_LEFT
            # reverse while turning left function
        elif current_state == State.REVERSE_LEFT:
            # reverse while turning left function
            trash = 0
        elif current_state == State.TURN_RIGHT or current_state == State.REVERSE_RIGHT:
            current_state == State.REACH_AND_TURN
            orig = True
            # call reach and turn function
        enforce_time =  curr_time + delay
    elif left_sense_sum > low_force_thresh:
        if current_state == State.GO_STRAIGHT:
            if left_sense_sum > high_force_thresh:
                current_state = State.REVERSE_RIGHT
                # reverse while turning right function
            else:
                current_state = State.TURN_RIGHT
                bias = np.array([0, -1])
        elif current_state == State.TURN_LEFT or current_state == State.REVERSE_LEFT:
            current_state == State.REACH_AND_TURN
            # call reach and turn function
        elif current_state == State.TURN_RIGHT:
            current_state = State.REVERSE_RIGHT
            # reverse while turning right function
        elif current_state == State.REACH_AND_TURN:
            # reverse while turning left function
            trash = 0
        enforce_time =  curr_time + delay
    elif curr_time >= enforce_time:
        current_state = State.GO_STRAIGHT
        bias = np.array([0,0])
    
    control_signal = np.array([-1, -1]) + bias

    obs, reward, terminated, truncated, info = nmf.step(control_signal,orig)
    nmf.render()

nmf.save_video("./outputs/pillars.mp4")
Video("./outputs/pillars.mp4")

  0%|          | 0/20000 [00:00<?, ?it/s]

 73%|███████▎  | 14672/20000 [02:38<00:57, 92.79it/s] 


PhysicsError: Physics state is invalid. Warning(s) raised: mjWARN_BADQACC

## New State Machine with outer layer being States

In [6]:
from enum import Enum, auto

# random state seed for reproducibility
seed = 1

class State(Enum):
    GO_STRAIGHT = 1
    TURN_LEFT = 2
    TURN_RIGHT = 3
    REVERSE = 4
    REACH_AND_TURN = 5

# Example of setting a current state
current_state = State.GO_STRAIGHT
next_state = State.GO_STRAIGHT

decision_interval = 0.2
run_time = 1
num_decision_steps = int(run_time / decision_interval)
physics_steps_per_decision_step = int(decision_interval / sim_params.timestep)

low_force_thresh = 2
high_force_thresh = 5
enforce_time = 0
turn_delay = 0.2
reverse_delay = 0.1

obs_hist = []
odor_history = []
obs, _ = nmf.reset(seed)
bias = np.array([0,0])
for i in trange(int(run_time / nmf.sim_params.timestep)):
    curr_time = i * nmf.sim_params.timestep
    left_sense = np.array(obs["contact_forces"][:5, 0:2])
    right_sense = np.array(obs["contact_forces"][18:23, 0:2])
    left_sense_sum = left_sense.sum()
    right_sense_sum = right_sense.sum()
    orig =False

    if current_state == State.GO_STRAIGHT:
        control_signal = np.array([1, 1])
        orig = False
        if right_sense_sum > high_force_thresh or left_sense_sum > high_force_thresh:
            current_state = State.REVERSE
            next_state = State.TURN_LEFT
            enforce_time = curr_time + reverse_delay
        elif left_sense_sum > high_force_thresh:
            current_state = State.REVERSE
            next_state = State.TURN_RIGHT
            enforce_time = curr_time + reverse_delay
        elif right_sense_sum > low_force_thresh:
            current_state = State.TURN_LEFT
            enforce_time = curr_time + turn_delay
        elif left_sense_sum > low_force_thresh:
            current_state = State.TURN_RIGHT
            enforce_time = curr_time + turn_delay
    elif current_state == State.TURN_RIGHT:
        control_signal = np.array([1, 0])
        orig = False
        if left_sense_sum > high_force_thresh:
            current_state = State.REVERSE
            enforce_time = curr_time + reverse_delay
        elif right_sense_sum > low_force_thresh:
            current_state = State.REACH_AND_TURN
        elif left_sense_sum > low_force_thresh:
            enforce_time = curr_time + turn_delay
        elif curr_time >= enforce_time:
            current_state = State.GO_STRAIGHT
    elif current_state == State.TURN_LEFT:
        control_signal = np.array([0, 1])
        orig = False
        if right_sense_sum > high_force_thresh:
            current_state = State.REVERSE
            enforce_time = curr_time + reverse_delay
        elif left_sense_sum > low_force_thresh:
            current_state = State.REACH_AND_TURN
        elif right_sense_sum > low_force_thresh:
            enforce_time = curr_time + turn_delay
        elif curr_time >= enforce_time:
            current_state = State.GO_STRAIGHT
    elif current_state == State.REVERSE:
        # control_signal = reverse_control_signal
        orig = True
        if curr_time >= enforce_time:
            current_state = next_state
    elif current_state == State.REACH_AND_TURN:
        # control_signal = reach and turn control signal
        orig = True
        # keep reaching and turning
        # if free path is found
        #   go straight
        trash=0

    obs, reward, terminated, truncated, info = nmf.step(control_signal, orig)
    nmf.render()

nmf.save_video("./outputs/pillars.mp4")
Video("./outputs/pillars.mp4")

100%|██████████| 10000/10000 [00:52<00:00, 191.64it/s]
